# Alura

## Inicialização

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [2]:
with open("artigos.txt", "r", encoding = "utf-8") as file:
    artigos = file.read()

## Analisando o corpus

In [3]:
artigos[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

In [4]:
tokens = artigos.split()
print(len(tokens))

416903


## Tokenização

In [5]:
tokens_artigo = nltk.tokenize.word_tokenize(artigos)
print(len(tokens_artigo))
print(tokens_artigo[:15])

515905
['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', ':', 'java', 'Para']


## Mantendo apenas palavras

In [6]:
tokens_artigo = list(filter(lambda x: x.isalpha(), tokens_artigo))
print(len(tokens_artigo))
print(tokens_artigo[:15])

403104
['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'Para', 'salvar']


## Normalização

In [7]:
tokens_normalizados = list(map(lambda x: x.lower(), tokens_artigo))
print(tokens_normalizados[:15])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'para', 'salvar']


In [8]:
tokens_unicos = list(set(tokens_normalizados))
print(len(tokens_unicos))
print(tokens_unicos[:15])
vocabulario = tokens_unicos[::]

18465
['liam', 'aproveita', 'freelances', 'overflow', 'preciso', 'inventado', 'desvalorizar', 'pertinente', 'gratuitos', 'arredonda', 'supera', 'personalizável', 'franca', 'jornada', 'strftime']


## Gerador de palavras

In [9]:
import string

def fatiar_palavra(palavra):
    fatiacoes = []
    for i in range(len(palavra)+1):
        fatiacoes.append( (palavra[:i], palavra[i:]) )
    return fatiacoes

def inserir_caractere(esq, dir):
    novas_palavras = []
    for c in 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç':
        novas_palavras.append(esq + c + dir)
    return novas_palavras

def gerador_palavras(palavra):
    todas_combinacoes = []
    for esq, dir in fatiar_palavra(palavra):
        todas_combinacoes.extend( inserir_caractere(esq, dir) )
    return todas_combinacoes

In [10]:
gerador_palavras("a")[:10]

['aa', 'ba', 'ca', 'da', 'ea', 'fa', 'ga', 'ha', 'ia', 'ja']

## Corretor

In [11]:
frequencia = nltk.FreqDist(tokens_normalizados)
total_palavras = len(tokens_normalizados)
frequencia

FreqDist({'de': 15502, 'o': 14056, 'que': 12230, 'a': 11099, 'e': 10501, 'para': 7710, 'um': 6368, 'é': 5899, 'uma': 5220, 'do': 5124, ...})

In [12]:
def probabilidade_palavra(palavra_gerada):
    return frequencia[palavra_gerada] / total_palavras

In [13]:
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key = probabilidade_palavra)
    return palavra_correta

In [14]:
probabilidade_palavra("lógica")

0.00023815194093831864

In [15]:
corretor("e")

'de'

## Avaliando o modelo

In [16]:
with open("palavras.txt", "r", encoding = "utf-8") as file:
    erradas = []
    corretas = []
    for line in file:
        correta, errada = (line.strip().split(maxsplit = 1))
        erradas.append(errada)
        corretas.append(correta)
    erradas = np.array(erradas)
    corretas = np.array(corretas)

In [17]:
def avaliador(corretas, erradas):
    total = corretas.shape[0]
    acertos = 0
    for i in range(total):
        acertos += corretor(erradas[i]) == corretas[i]
    print("Taxa de acertos = {:.2f}%".format(acertos * 100 / total))

avaliador(corretas, erradas)

Taxa de acertos = 1.08%


## Adicionando operação de Delete

In [18]:
def delete_palavra(palavra):
    novas_palavras = []
    for i in range(len(palavra)):
        novas_palavras.append( palavra[:i] + palavra[i+1:] )
    return novas_palavras

delete_palavra("teste")

['este', 'tste', 'tete', 'tese', 'test']

In [19]:
def gerador_palavras(palavra):
    todas_combinacoes = []
    for esq, dir in fatiar_palavra(palavra):
        todas_combinacoes.extend( inserir_caractere(esq, dir) )
    todas_combinacoes.extend( delete_palavra(palavra) )
    return todas_combinacoes

In [20]:
avaliador(corretas, erradas)

Taxa de acertos = 41.40%


## Operação de Substituir

In [21]:
def substituir_palavra(palavra):
    novas_palavras = []
    for i in range(len(palavra)):
        for c in 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç':
            novas_palavras.append(palavra[:i] + c + palavra[i+1:])
    return novas_palavras

print(substituir_palavra("ab"))

['ab', 'bb', 'cb', 'db', 'eb', 'fb', 'gb', 'hb', 'ib', 'jb', 'kb', 'lb', 'mb', 'nb', 'ob', 'pb', 'qb', 'rb', 'sb', 'tb', 'ub', 'vb', 'wb', 'xb', 'yb', 'zb', 'àb', 'áb', 'âb', 'ãb', 'èb', 'éb', 'êb', 'ìb', 'íb', 'îb', 'òb', 'ób', 'ôb', 'õb', 'ùb', 'úb', 'ûb', 'çb', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as', 'at', 'au', 'av', 'aw', 'ax', 'ay', 'az', 'aà', 'aá', 'aâ', 'aã', 'aè', 'aé', 'aê', 'aì', 'aí', 'aî', 'aò', 'aó', 'aô', 'aõ', 'aù', 'aú', 'aû', 'aç']


In [22]:
def gerador_palavras(palavra):
    todas_combinacoes = []
    for esq, dir in fatiar_palavra(palavra):
        todas_combinacoes.extend( inserir_caractere(esq, dir) )
    todas_combinacoes.extend( delete_palavra(palavra) )
    todas_combinacoes.extend( substituir_palavra(palavra) )
    return todas_combinacoes

In [23]:
avaliador(corretas, erradas)

Taxa de acertos = 76.34%


## Inversão de letras

In [24]:
def inverter_palavra(palavra):
    novas_palavras = []
    for i in range(len(palavra) - 1):
        novas_palavras.append( palavra[:i] + palavra[i:i+2][::-1] + palavra[i+2:]  )
    return novas_palavras

inverter_palavra("lógica")

['ólgica', 'lgóica', 'lóigca', 'lógcia', 'lógiac']

In [25]:
def gerador_palavras(palavra):
    todas_combinacoes = []
    for esq, dir in fatiar_palavra(palavra):
        todas_combinacoes.extend( inserir_caractere(esq, dir) )
    todas_combinacoes.extend( delete_palavra(palavra) )
    todas_combinacoes.extend( substituir_palavra(palavra) )
    todas_combinacoes.extend( inverter_palavra(palavra) )
    return todas_combinacoes

In [26]:
avaliador(corretas, erradas)

Taxa de acertos = 76.34%


## Palavras desconhecidas ao vocabulário

In [27]:
desconhecidas = 0
for palavra in corretas:
    if palavra not in vocabulario:
        desconhecidas += 1
print("{} palavras desconhecidas de {}".format( desconhecidas , len(corretas) ) )

13 palavras desconhecidas de 186


In [28]:
print("Porcentagem de erro por palavras desconhecidas é: {:.2f}%".format( (13 / 186) * 100) )

Porcentagem de erro por palavras desconhecidas é: 6.99%


In [29]:
def avaliador(corretas, erradas):
    total = corretas.shape[0]
    acertos = 0
    for i in range(total):
        acertos += corretor(erradas[i]) == corretas[i]
    print("Taxa de acertos = {:.2f}%".format(acertos * 100 / total))

    desconhecidas = 0
    for palavra in corretas:
        if palavra not in vocabulario:
            desconhecidas += 1

    print("Porcentagem de erro por palavras desconhecidas é: {:.2f}%".format( (desconhecidas / len(corretas)) * 100) )

avaliador(corretas, erradas)

Taxa de acertos = 76.34%
Porcentagem de erro por palavras desconhecidas é: 6.99%


## Gerador turbinado

In [30]:
def gerador_turbinado(palavras, iteracoes = 2):
    if type(palavras) == str or type(palavras) == np.str_: palavras = [palavras]

    if iteracoes == 0: 
        return palavras
    todas_palavras = []
    for palavra in palavras:
        todas_palavras.extend( gerador_palavras(palavra) )
    return gerador_turbinado( set(todas_palavras), iteracoes - 1)

In [31]:
def corretor_turbinado(palavra, iteracoes = 2):
    palavras_geradas = gerador_turbinado(palavra, iteracoes)
    palavra_correta = max(palavras_geradas, key = probabilidade_palavra)
    return palavra_correta

In [32]:
corretor_turbinado("logicaa", iteracoes = 2)

'lógica'

In [33]:
def avaliador_turbinado(corretas, erradas, iteracoes = 2):
    total = corretas.shape[0]
    acertos = 0
    for i in range(total):
        acertos += corretor_turbinado(erradas[i], iteracoes) == corretas[i]
    print("Taxa de acertos = {:.2f}%".format(acertos * 100 / total))

    desconhecidas = 0
    for palavra in corretas:
        if palavra not in vocabulario:
            desconhecidas += 1

    print("Porcentagem de erro por palavras desconhecidas é: {:.2f}%".format( (desconhecidas / len(corretas)) * 100) )


avaliador_turbinado(corretas, erradas, iteracoes = 2)

Taxa de acertos = 54.84%
Porcentagem de erro por palavras desconhecidas é: 6.99%


In [37]:
len(gerador_turbinado("aaaa"))

58098

## Analisando os erros

In [39]:
tabela_resultados = []
for i in range(len(corretas)):
    resposta_corretor = corretor(erradas[i])
    resposta_corretor_turbinado = corretor_turbinado(erradas[i])
    tabela_resultados.append( [erradas[i], corretas[i], resposta_corretor, resposta_corretor_turbinado, corretas[i] == resposta_corretor, corretas[i] == resposta_corretor_turbinado] )
df_resultados = pd.DataFrame(tabela_resultados, columns = ["Erradas", "Corretas", "Corretor", "Corretor_Turbinado", "Corretor_Acerto", "Corretor_Turbinado_Acerto"])
df_resultados

,Erradas,Corretas,Corretor,Corretor_Turbinado,Corretor_Acerto,Corretor_Turbinado_Acerto
0,pyodemos,podemos,podemos,podemos,True,True
1,esje,esse,esse,se,True,False
2,jrá,já,já,já,True,True
3,nossov,nosso,nosso,nosso,True,True
4,sãêo,são,são,não,True,False
...,...,...,...,...,...,...
181,provalecer,prevalecer,aprovalecer,prevalece,False,False
182,esteje,esteja,esteja,este,True,False
183,mindigo,mendigo,amindigo,indico,False,False
184,célebro,cérebro,cérebro,cérebro,True,True


# Ideias

Ideias para melhorar o corretor:

- Análise com base no contexto
- Melhorar a base de dados
- Otimizar o gerador turbinado (memoização)